In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
from tensorflow import keras
import json
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import dataset
from util import (
    get_place_to_index_mapping,
    get_incident_to_index_mapping
)
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Flatten, Permute
from keras import Sequential
import keras.backend as kb
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import PIL

In [3]:
abs_path = "/kuacc/users/asafaya19/cv-project"
train_json = os.path.join(abs_path ,"eccv_train.json")
val_json = os.path.join(abs_path ,"eccv_val.json")
data_dir = os.path.join(abs_path, "data")
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")

train_paths = json.loads(open(train_json).readline())
val_paths = json.loads(open(val_json).readline())

place_to_idx = get_place_to_index_mapping()
incident_to_idx = get_incident_to_index_mapping()

In [4]:
def get_dataset(paths,file_dir, threshold=1000):
    train_set = []
    for path in tqdm(paths, leave=False):
    #     print(path)
        if not os.path.exists(os.path.join(file_dir, path)):
            continue
        # Make sure image is not corrupt, try importing it
        try:
            img = PIL.Image.open(os.path.join(file_dir, path))
            img.resize((224, 224))
        except:
            continue
        nump = len(place_to_idx) + 1
        numi = len(incident_to_idx) + 1
        place_labels = np.zeros(nump, np.float32)
        place_weights = np.zeros(nump, np.float32)
        incident_labels = np.zeros(numi, np.float32)
        incident_weights = np.zeros(numi, np.float32)

        incidents = paths[path]["incidents"]
        for k in incidents:
            lbl = incidents[k]
            if lbl==1:
                # We are sure this instance is only this incident
                incident_labels[incident_to_idx[k]]=1
                incident_weights = np.ones(numi, np.float32)
            else:
                # We are only sure that this image is not that incident
                incident_weights[incident_to_idx[k]]=1
        if len(incidents)==0:
            # No incident
            incident_labels[-1]=1
            incident_weights = np.ones(numi, np.float32)

        places = paths[path]["places"]
        for k in places:
            lbl = places[k]
            if lbl==1:
                # We are sure this instance is only this incident
                place_labels[place_to_idx[k]]=1
                place_weights = np.ones(nump, np.float32)
            else:
                # We are only sure that this image is not that incident
                place_weights[place_to_idx[k]]=1
        if len(places)==0:
            # No place
            place_labels[-1]=1
            place_weights = np.ones(nump, np.float32)


        train_set.append({
            "path":path,
            "incident_labels":incident_labels,
            "incident_weights":incident_weights,
            "incidents":np.vstack((incident_labels, incident_weights)),
            "place_labels":place_labels,
            "place_weights":place_weights,
            "place":np.vstack((place_labels, place_weights))
        })
        if len(train_set)>=threshold:
            break
    return train_set

In [16]:
def getpreprocessfunc():
    mean = np.asarray([0.485, 0.456, 0.406]).reshape(3, 1, 1).astype(np.float32)
    std = np.asarray([0.229, 0.224, 0.225]).reshape(3, 1, 1).astype(np.float32)
    def preprocessfunc(img):
#         print(img.shape)
#         print(type(img))
        img /= 255
        img -= mean
        img /= std
        return img
    return preprocessfunc

In [17]:
# Enclosure to retain state
def get_weighted_accuracy():
    m = keras.metrics.CategoricalAccuracy()
    def weighted_accuracy(y_true, y_preds):
        y_true = tf.reshape(y_true, (bs, 2, -1))
        y_true_lbls = y_true[:,0,:]
        return m(y_true_lbls, y_preds)
    return weighted_accuracy

In [18]:
def weighted_loss(y_true, y_preds):
    bce = keras.losses.BinaryCrossentropy(keras.losses.Reduction.NONE)
#     tf.print(y_true)
#     tf.print(y_preds)
#     print(y_true.shape)
#     print(y_preds.shape)
    bs = y_true.shape[0]
    
    print(y_true.shape, y_preds.shape)
    
    y_true = tf.reshape(y_true, (bs, 2, -1))
    y_true_lbls = y_true[:,0,:]
    y_true_weights = y_true[:,1,:]
    bce_loss = bce(y_true_lbls, y_preds)
#     import pdb
#     pdb.set_trace()
#     return bce_loss
    return tf.reduce_sum(tf.multiply(bce_loss, y_true_weights))

In [19]:
from keras.models import model_from_json


class FinalModel(keras.Model):
    def __init__(self, trunk_model, incident_weights, place_weights):
        super(FinalModel, self).__init__()
        self.permute = Permute((2, 3, 1))
        self.cropped = keras.layers.experimental.preprocessing.RandomCrop(224, 224)
        self.permuteback = Permute((3, 1, 2))
        self.trunk_model = trunk_model
        self.incident_proj = Dense(len(incident_to_idx), input_shape=(1024,), name="incidents_projection", weights=incident_weights)
        self.places_proj = Dense(len(place_to_idx), input_shape=(1024,), name="places_projection", weights=place_weights)
        
    def call(self, inputs):
        x = self.permute(inputs)
        x = self.cropped(x)
        x = self.permuteback(x)
        x = self.trunk_model(x)
        
        return self.incident_proj(x), self.places_proj(x)

In [11]:
train_set = get_dataset(train_paths, train_dir, 1000)
val_set = get_dataset(val_paths, val_dir)

train_df = pd.DataFrame(train_set)
val_df = pd.DataFrame(val_set)

In [20]:
imgen = ImageDataGenerator(
    horizontal_flip=True,
    preprocessing_function=getpreprocessfunc(),
)

imgen = imgen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col="path",
    y_col=["incidents", "place"],
    weight_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="multi_output",
    batch_size=64,
    shuffle=False,
    seed=True,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset=None,
    interpolation="nearest",
    validate_filenames=True,
)

Found 1000 validated image filenames.


In [21]:
losses = {
    "output_1": weighted_loss,
    "output_2": weighted_loss,
}

In [22]:
opt = keras.optimizers.Adam(lr=1e-5)

trunk_model = model_from_json(open(os.path.join(abs_path,'models/trunk.json'), 'r').read())
trunk_model.load_weights(os.path.join(abs_path,"models/trunk.h5"))
place_w = np.load(os.path.join(abs_path,'models/place_w.npy')).T
place_b = np.load(os.path.join(abs_path,'models/place_b.npy')).T
incident_w = np.load(os.path.join(abs_path,'models/incident_w.npy')).T
incident_b = np.load(os.path.join(abs_path,'models/incident_b.npy')).T

mdl = FinalModel(trunk_model, [incident_w, incident_b], [place_w, place_b])

mdl.compile(optimizer=opt, loss=losses, metrics=[get_weighted_accuracy()])

In [23]:
x, y = next(imgen)
o = mdl(x)

In [24]:
from functools import partial
import sklearn
from sklearn.metrics._base import type_of_target

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def fixed_precision_recall_curve(y_true, probas_pred, *, pos_label=None,
                           sample_weight=None):

    fps, tps, thresholds = sklearn.metrics._ranking._binary_clf_curve(y_true, probas_pred,
                                             pos_label=pos_label,
                                             sample_weight=sample_weight)

    precision = tps / (tps + fps)
    precision[np.isnan(precision)] = 0
    recall = np.ones(tps.size) if tps[-1] == 0 else tps / tps[-1]

    # stop when full recall attained
    # and reverse the outputs so recall is decreasing
    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)
    return np.r_[precision[sl], 1], np.r_[recall[sl], 0], thresholds[sl]


def average_precision_score(y_true, y_score, *, average="macro", pos_label=1,
                            sample_weight=None):
    def _binary_uninterpolated_average_precision(
            y_true, y_score, pos_label=1, sample_weight=None):
        precision, recall, _ = fixed_precision_recall_curve(
            y_true, y_score, pos_label=pos_label, sample_weight=sample_weight)
        # Return the step function integral
        # The following works because the last entry of precision is
        # guaranteed to be 1, as returned by precision_recall_curve
        return -np.sum(np.diff(recall) * np.array(precision)[:-1])

    y_type = type_of_target(y_true)
    if y_type == "multilabel-indicator" and pos_label != 1:
        raise ValueError("Parameter pos_label is fixed to 1 for "
                         "multilabel-indicator y_true. Do not set "
                         "pos_label or set pos_label to 1.")
    elif y_type == "binary":
        # Convert to Python primitive type to avoid NumPy type / Python str
        # comparison. See https://github.com/numpy/numpy/issues/6784
        present_labels = np.unique(y_true).tolist()
        if len(present_labels) == 2 and pos_label not in present_labels:
            raise ValueError(
                f"pos_label={pos_label} is not a valid label. It should be "
                f"one of {present_labels}"
            )
    average_precision = partial(_binary_uninterpolated_average_precision,
                                pos_label=pos_label)
    return sklearn.metrics._base._average_binary_score(average_precision, y_true, y_score,
                                 average, sample_weight=sample_weight)

In [33]:
val_set = get_dataset(val_paths, val_dir, threshold=100000)
val_df = pd.DataFrame(val_set)

imgen = ImageDataGenerator(
    preprocessing_function=getpreprocessfunc(),
)

imgen = imgen.flow_from_dataframe(
    val_df,
    directory=val_dir,
    x_col="path",
    y_col=["incidents", "place"],
    weight_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="multi_output",
    batch_size=64,
    shuffle=False,
    seed=True,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset=None,
    interpolation="nearest",
    validate_filenames=True,
)

  3%|▎         | 1856/57207 [00:37<23:43, 38.89it/s]/kuacc/users/asafaya19/anaconda3/envs/ml-graphs/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:792: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
 12%|█▏        | 6622/57207 [02:21<13:09, 64.05it/s]  /kuacc/users/asafaya19/anaconda3/envs/ml-graphs/lib/python3.7/site-packages/PIL/Image.py:2837: DecompressionBombWarning: Image size (134599200 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 41%|████▏     | 23642/57207 [08:55<09:07, 61.26it/s]  /kuacc/users/asafaya19/anaconda3/envs/ml-graphs/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 9 bytes but only got 8. Skipping tag 33432
  "Possibly corrupt EXIF data.  "
 57%|█████▋    | 32355/57207 [12:13<10:01, 41.30it/s]/kuacc/users/asafaya19/anaconda3/envs/ml-graphs/lib/python3.7/site-packages/PIL/TiffIm

Found 47649 validated image filenames.


In [25]:
imgen.reset()

nbatches = 0
imap = 0
pmap = 0

for x, y in tqdm(imgen):
    
    y1 = y[0]
    y2 = y[1]

    out = mdl.predict(x)
    
    iout = sigmoid(out[0])
    pout = sigmoid(out[1])
    
    itrue = y1[:,0,:-1]
    ptrue = y2[:,0,:-1]
    
    imap += average_precision_score(itrue, iout)
    pmap += average_precision_score(ptrue, pout)
    nbatches += 1
    
    if nbatches > np.ceil(val_df.shape[0] / 64):
        break
    
pmap.item() / nbatches, imap.item() / nbatches

100%|██████████| 16/16 [00:31<00:00,  1.97s/it]


(0.1065806187405204, 0.3706045208781187)